In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from tqdm import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nome_tentativo = ''
path_manual = '/content/drive/MyDrive/EIM/Retina/RETINA/Original'                 #Inserire percorso maschere manuali

path_result = '/content/drive/MyDrive/EIM/Retina/RETINA/Preprocessed_'+nome_tentativo               #Inserire percorso maschere

list_manual = os.path.listdir(path_manual)

In [ ]:
# Funzioni pre-process

def add_gaussian_noise(image, mean=0, sigma=25):
#Aggiunge rumore gaussiano all'immagine
    row, col, ch = image.shape
    noise = np.random.normal(mean, sigma, (row, col, ch))
    noisy_image = np.clip(image + noise, np.min(image), np.max(image))
    noisy_image = noisy_image.astype(np.uint8)
    return noisy_image

def extract_subarrays_and_coordinates(arr, size):
  #Questa funzione restituisce tre liste: la prima di patch dell'immagine, la seconda di coordinate x e
  #e la terza coordinate y
    arrays = []
    coord_x = []
    coord_y = []

    # Ottieni le dimensioni dell'array in ingresso
    rows, cols = arr.shape

    # Estrai gli array quadrati di dimensione size
    for i in range(0, rows, size):
        for j in range(0, cols, size):
            subarray = arr[i:i+size, j:j+size]

            # Verifica le dimensioni dell'array estratto
            if subarray.shape == (size, size):
                arrays.append(subarray)
                coord_x.append(j)
                coord_y.append(i)
            else:
                # Stampa un messaggio di errore se le dimensioni non sono corrette
                print(f"Errore: Array di dimensioni {subarray.shape} invece di ({size}, {size})."
                      f" Ignorato a coordinate ({i}, {j}).")

    return arrays, coord_x, coord_y

def resize_image(image_array, size):

    # Verifica se l'immagine è quadrata
    if image_array.shape[0] != image_array.shape[1]:
        raise ValueError("L'immagine deve essere quadrata.")

    # Dimensione dell'immagine originale
    original_size = image_array.shape[0]

    # Calcola il rapporto tra le dimensioni
    img_ratio = original_size / size

    # Crea l'immagine risultante con dimensioni size x size
    resized_image = np.zeros((size, size, image_array.shape[2]) if len(image_array.shape) == 3 else (size, size))
    if img_ratio > 1:
      # Itera attraverso le patch e assegna il pixel o il valore più scuro della patch all'immagine risultante
      for i in range(size):
          for j in range(size):
              # Calcola gli indici corrispondenti nell'immagine originale
              x_start = int(i * img_ratio)
              x_end = int((i + 1) * img_ratio)
              y_start = int(j * img_ratio)
              y_end = int((j + 1) * img_ratio)

              # Estrae la patch
              patch = image_array[x_start:x_end, y_start:y_end] if len(image_array.shape) == 2 else image_array[x_start:x_end, y_start:y_end, 0]

              dark_pixel = np.min(patch)

              # Assegna il pixel più scuro all'immagine risultante
              resized_image[i, j] = dark_pixel
    elif img_ratio < 1:
        for i in range(original_size):
          for j in range(original_size):
              # Calcola gli indici corrispondenti nell'immagine originale
              x_start = int(i / img_ratio)
              x_end = int((i + 1) / img_ratio)
              y_start = int(j / img_ratio)
              y_end = int((j + 1) / img_ratio)

              resized_image[x_start:x_end, y_start:y_end] = image_array[i,j]
    else:
      resized_image = image_array
    return resized_image


[[ 1.6733334   0.93571267  0.09265563 -1.72979489]
 [-0.72774301 -0.06833142  1.74135743  1.3247553 ]
 [ 0.54510722 -0.86866785 -0.9111401  -0.07062113]
 [-1.5695633   1.40852085  2.18225041  1.48198973]]


In [ ]:
for i in tqdm(range(len(list_manual))):
  pil_image = Image.open(list_manual[i])
  image_array = np.array(pil_image)
  pic_save = image_array.save(os.path.join(path_result,list_manual[i]))